## Dirac Notation

A **qubit** is a "quantum bit" that owns a state of either 0 or 1. Unlike in the classical interpretaion that the bit should be either 0 or 1 exclusively at a time, a qubit can be "partially" 0 and partially 1. We specify the "partially" by a complex number called **amplitude**. For example, $|\psi\rangle = \sqrt{0.2}|{0}\rangle + i\sqrt{0.8}|1\rangle$ represents a legal state of a qubit, where $\sqrt{0.2}$ and $i\sqrt{0.8}$ are the amplitudes of state $0$ and $1$, respectively. We call this "$|\rangle$" the **dirac notation**, which is widely used in quantum mechanics.

Here, the certificate for a legal qubit is that the sqaures of the amplitudes sum up to $1$. There will be no surprise if you know that these factors are essentially the square roots of the probability of each state. 

If the state is represented by multiple qubits, we need more factors to describe the probability distribution. In gerneral it's $2^w$ factors for a $w$-qubit state. For example:

$$
    \alpha|00\rangle + \beta|01\rangle + \gamma|10\rangle + \delta|11\rangle \quad \alpha, \beta, \gamma, \delta \in \mathbb{C}
$$

is the general form for 2-qubit quantum states. A concrete example may be $\sqrt{0.1}|00\rangle + \sqrt{0.4}|01\rangle -\sqrt{0.5}|11\rangle$. For simplicity (arguable), we can also write the bits in base 10, in which case we will have $\sqrt{0.1}|0\rangle + \sqrt{0.4}|1\rangle - \sqrt{0.5}|3\rangle$ for the same example.

If we already have two wires each carrying one qubit of states $\psi_1$ and $\psi_2$, how can we get the overall state? i.e. how to get the 4 factors out of the original ones? Say $\psi_1 = \sqrt{0.2}|0\rangle + \sqrt{0.8}|1\rangle$, $\psi_2 = \sqrt{0.4}|0\rangle + \sqrt{0.6}|1\rangle$, the tensor product is helpful here:

$$
\begin{align*}
    \psi &= \psi_1 \otimes \psi_2 \\
    &= (\sqrt{0.2}|0\rangle + \sqrt{0.8}|1\rangle)\otimes(\sqrt{0.4}|0\rangle + \sqrt{0.6}|1\rangle) \\
    &= \sqrt{0.2\times0.4}|00\rangle + \sqrt{0.2\times0.6}|01\rangle + \sqrt{0.8\times0.4}|10\rangle + \sqrt{0.8\times0.6}|11\rangle \\
    &= \sqrt{0.08}|00\rangle + \sqrt{0.12}|01\rangle + \sqrt{0.32}|10\rangle + \sqrt{0.48}|11\rangle
\end{align*}
$$

Note that *not* all states can be derived from tensor products. States that can be represented as a tensor product are called **product states**. Others are called **entangled states**. An example of the latter is $\psi = \frac{1}{\sqrt{2}}|01\rangle + \frac{1}{\sqrt{2}}|10\rangle$, in which case the measured results of the qubits are dependent on each other (say we get 1 for the first qubit, then it's sure the other qubit is 0).

## Vector Notation

Another notation for quantum states is the **vector notation**. Since conventially, each possible state are listed in the increasing order, we can just keep the amplitudes of each state and write them in the vector form. For example, $\psi = \sqrt{0.2}|0\rangle + \sqrt{0.8}1\rangle$ can be equivalently written as:

$$
    \psi = 
    \begin{bmatrix}
        \sqrt{0.2} \\ \sqrt{0.8}
    \end{bmatrix}
$$

Note that in this notation we cannot omit the items with 0 amplitude.

It's natural to consider each possible state as a basic vector, so that the total state can be represented by the weighed sum of these vectors, just like how we interpret a vector in $\mathbb{R}^k$. For example, for a $w$-qubit state where $w = 2$, we have the following basic vectors in $\mathbb{R}^{2^w}$:

$$
    |00\rangle =
    \begin{bmatrix}
        1 \\ 0 \\ 0 \\ 0
    \end{bmatrix}
    \qquad
    |01\rangle =
    \begin{bmatrix}
        0 \\ 1 \\ 0 \\ 0
    \end{bmatrix}
    \qquad
    |10\rangle =
    \begin{bmatrix}
        0 \\ 0 \\ 1 \\ 0
    \end{bmatrix}
    \qquad
    |11\rangle =
    \begin{bmatrix}
        0 \\ 0 \\ 0 \\ 1
    \end{bmatrix}
$$

Then, a state $\psi = \sqrt{0.08}|00\rangle + \sqrt{0.12}|01\rangle + \sqrt{0.32}|10\rangle + \sqrt{0.48}|11\rangle$ can be written in the following vector form:

$$
\begin{align*}
    \psi &= \sqrt{0.08}|00\rangle + \sqrt{0.12}|01\rangle + \sqrt{0.32}|10\rangle + \sqrt{0.48}|11\rangle \\
    &=
    \sqrt{0.08}\,
    \begin{bmatrix}
        1 & 0 & 0 & 0
    \end{bmatrix}^T + 
    \sqrt{0.12}\,
    \begin{bmatrix}
        0 & 1 & 0 & 0
    \end{bmatrix}^T + 
    \sqrt{0.32}\,
    \begin{bmatrix}
        0 & 0 & 1 & 0
    \end{bmatrix}^T + 
    \sqrt{0.48}\,
    \begin{bmatrix}
        0 & 0 & 0 & 1
    \end{bmatrix} \\
    &= 
    \begin{bmatrix}
        0.08 & 0.12 & 0.32 & 0.48
    \end{bmatrix}^T
\end{align*}
$$

Earlier we've said that the square sum of all amplitudes should be $1$, this can be formally expressed in mathematical form:

$$
    \langle \psi|\psi\rangle = |\psi\rangle(|\psi\rangle^T)^* = 1
$$


We will find vectors very helpful later when doing calculations.

## Simulating Qubits

We will use data structures in python to simulate qubits. Let's first import some useful packages:

In [45]:
import math
import numpy

A state can be expressed in the following form:

In [2]:
state = [
    (numpy.sqrt(0.1), '00'),
    (numpy.sqrt(0.4), '01'),
    (numpy.sqrt(0.5), '11')
]

The following function is used to print a state in the dirac notation, one in binary and one in decimal

In [13]:
def print_dirac_binary(state):
    result = '('
    for (p, bits) in state:
        result += str(p) + ' |' + bits + '> + '
    result = result[:-3] + ')'
    print(result)

def print_dirac_decimal(state):
    def binary_to_decimal(binary):
        result = 0
        for d in binary:
            result = result * 2 + int(d)
        return str(result)
    result = '('
    for (p, bits) in state:
        result += str(p) + ' |' + binary_to_decimal(bits) + '> + '
    result = result[:-3] + ')'
    print(result)

Here's the effect:

In [14]:
print_dirac_binary(state)
print_dirac_decimal(state)

(0.31622776601683794 |00> + 0.6324555320336759 |01> + 0.7071067811865476 |11>)
(0.31622776601683794 |0> + 0.6324555320336759 |1> + 0.7071067811865476 |3>)


We may also want to convert between states and their vector forms:

In [76]:
def state_to_vector(state):
    limit = 2 ** len(state[0][1])
    result = numpy.zeros(limit)
    j = 0
    for i in range(0, limit):
        (p, bits) = state[j]
        if i == int(bits, 2):
            result[i] = p
            j += 1
        else:
            result[i] = 0.0
    return result

def vector_to_state(vec):
    def decimal_to_binary(decimal, length):
        result = bin(decimal)[2:]
        return '0'*(length - len(result)) + result
    length = math.frexp(len(vec))[1] - 1  
    return [(p, decimal_to_binary(i, length)) for i, p in enumerate(vec)]

Let's see what we've got:

In [78]:
print(state_to_vector(state))
print_dirac_binary(vector_to_state(state_to_vector(state)))

[0.31622777 0.63245553 0.         0.70710678]
(0.31622776601683794 |00> + 0.6324555320336759 |01> + 0.0 |10> + 0.7071067811865476 |11>)


## Quantum Gates

Just like logic gates for classical system, there are **quantum gates** that modify the states of a quantum system. Let's introduce some of them:

### Hadamard

The Hadamard gate operates on a single qubit with the following mapping rule:

$$
    |0\rangle \mapsto \frac{1}{\sqrt{2}}|0\rangle + \frac{1}{\sqrt{2}}|1\rangle
    \qquad
    |1\rangle \mapsto \frac{1}{\sqrt{2}}|0\rangle - \frac{1}{\sqrt{2}}|1\rangle
$$

As usual, it can be represented by a matrix:

$$
    H = \frac{1}{\sqrt{2}}
    \begin{bmatrix}
        1 & 1 \\ 1 & -1
    \end{bmatrix}
$$

For a 1-qubit state, we can apply $H$ to it to get the state after the Hadamard gate. Say $\psi = \sqrt{0.5}|0\rangle + \sqrt{0.5}|1\rangle$, then:

$$
    H\psi = 
    \frac{1}{\sqrt{2}}
    \begin{bmatrix}
        1 & 1 \\ 1 & -1
    \end{bmatrix}
    \begin{bmatrix}
        \sqrt{0.5} \\ \sqrt{0.5}
    \end{bmatrix}
    = 
    \begin{bmatrix}
        1 \\ 0
    \end{bmatrix}
    = |0\rangle
$$

### Phase Shift Gate

The phase shift gate has the following rule:

$$
    |0\rangle \mapsto |0\rangle
    \qquad
    |1\rangle \mapsto e^{i\theta}|1\rangle
$$

Here $\theta$ is some given value. The matrix form is as follows:

$$
    P(\theta) =
    \begin{bmatrix}
        1 & 0 \\ 0 & e^{i\theta}
    \end{bmatrix}
$$

### CNOT Gate

The CNOT gate is a controlled gate that takes two qubits and performs NOT operation on the second qubit if the first qubit is 1:

$$
    |00\rangle \mapsto |00\rangle
    \qquad
    |01\rangle \mapsto |01\rangle
    \qquad
    |10\rangle \mapsto |11\rangle
    \qquad
    |11\rangle \mapsto |10\rangle
$$

The matrix form of the CNOT gate is:

$$
    CNOT =
    \begin{bmatrix}
        1 & 0 & 0 & 0 \\
        0 & 1 & 0 & 0 \\
        0 & 0 & 0 & 1 \\
        0 & 0 & 1 & 0
    \end{bmatrix}
$$

The three gates introduced above will be our **universal gate set**, like the NAND and NOR gates among the classical gates. In other words, we can use these three gates to perform anything in quantum computing. All the gates we will learn later can be constructed from the universal gates.

We can simulate these gates by defining functions:

10

In [44]:
bin(10)

'0b1010'